# Land Cover Raster Data Exploration

In [1]:
from osgeo import gdal
from osgeo import gdal_array
import numpy as np
import rasterio
#from affine import Affine
import pyproj

Read in and open `.img` file.

In [2]:
dataset = gdal.Open('../../../zip_extract/Land_Cover/NYC_2017_LiDAR_LandCover.img')

## Dataset Info

In [3]:
print('Metadata:',dataset.GetMetadata())
print("Driver: {}/{} \n".format(dataset.GetDriver().ShortName,
                            dataset.GetDriver().LongName))
print("Size is {} x {} x {} \n".format(dataset.RasterXSize,
                                    dataset.RasterYSize,
                                    dataset.RasterCount))
print("Projection is {} \n".format(dataset.GetProjection()))
geotransform = dataset.GetGeoTransform()
if geotransform:
    print(f"Row Rotation: {geotransform[2]}")
    print(f"Column Rotation: {geotransform[4]}")
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

Metadata: {}
Driver: HFA/Erdas Imagine Images (.img) 

Size is 310844 x 314414 x 1 

Projection is PROJCS["NAD83 New York State Planes, Long Island, US Foot",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",40.1666666666667],PARAMETER["central_meridian",-74],PARAMETER["standard_parallel_1",41.0333333333333],PARAMETER["standard_parallel_2",40.6666666666667],PARAMETER["false_easting",984250],PARAMETER["false_northing",0],UNIT["us_survey_feet",0.304800609601219],AXIS["Easting",EAST],AXIS["Northing",NORTH]] 

Row Rotation: 0.0
Column Rotation: 0.0
Origin = (912286.93, 273618.3)
Pixel Size = (0.5, -0.5)


Row and column rotations of 0 mean the image is north up. The pixel size is 0.5ft x 0.5ft.

In [4]:
band = dataset.GetRasterBand(1)
print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))
min = band.GetMinimum()
max = band.GetMaximum()
if not min or not max:
    (min,max) = band.ComputeRasterMinMax(True)
print("Min={:.3f}, Max={:.3f}".format(min,max))
if band.GetOverviewCount() > 0:
    print("Band has {} overviews".format(band.GetOverviewCount()))
if band.GetRasterColorTable():
    print("Band has a color table with {} entries".format(band.GetRasterColorTable().GetCount()))
print('Size is {} x {}.'.format(band.XSize, band.YSize))

Band Type=Byte
Min=1.000, Max=8.000
Band has 11 overviews
Size is 310844 x 314414.


In [5]:
stats = band.GetStatistics(False,True)
print(f'Min: {stats[0]}')
print(f'Max: {stats[1]}')
print(f'Mean: {stats[2]}')
print(f'Std Dev: {stats[3]}')

Min: 1.0
Max: 8.0
Mean: 4.1611546745131
Std Dev: 2.0263974871886


In [6]:
#number of pixels in each class
band.GetDefaultHistogram()

(-0.5,
 8.5,
 9,
 [0,
  344109811,
  253881597,
  26155236,
  611739712,
  310075293,
  213863594,
  376111202,
  11547201])

In [7]:
#tile size
band.GetBlockSize()

[64, 64]

In [8]:
# #read in entire img --> creates 97GB numpy array
# lc_array = gdal_array.BandReadAsArray(band)

## Getting latitude and longitude of center of each pixel

In [65]:
def get_lat_long_arrays(band, x_size, y_size, x_offset = 0, y_offset = 0, proj_from = pyproj.Proj(2263), proj_to = pyproj.Proj(proj = 'latlong')):
    
    lc_array = gdal_array.BandReadAsArray(band, x_offset, y_offset, x_size, y_size)
    
    cols, rows = np.meshgrid(np.arange(x_size), np.arange(y_size))
    
    rc_en = lambda r,c : (geotransform[0] + (x_offset + c + 0.5) * geotransform[1], geotransform[3] + (y_offset + r + 0.5) * geotransform[5])
    
    eastings, northings = np.vectorize(rc_en, otypes = [float, float])(rows, cols)
    
    transformer = pyproj.Transformer.from_proj(proj_from, proj_to)
    
    longs, lats = transformer.transform(eastings, northings)
    
    print(f'Land cover values in sample: {np.unique(lc_array)}')
    print(f'Area of sample: {x_size*y_size/4} sq ft')
    print(f'Total pixels (observations) in sample: {x_size*y_size}')

    return lats, longs

In [66]:
get_lat_long_arrays(BAND,2,2)

Land cover values in sample: [0]
Area of sample: 1.0 sq ft
Total pixels (observations) in sample: 4


(array([[40.91739774, 40.91739775],
        [40.91739637, 40.91739638]]),
 array([[-74.26037993, -74.26037812],
        [-74.26037992, -74.26037812]]))

In [130]:
get_lat_long_arrays(BAND,2,2,150000,200000)

Land cover values in sample: [6]
Area of sample: 1.0 sq ft
Total pixels (observations) in sample: 4


(array([[40.64321751, 40.64321751],
        [40.64321614, 40.64321614]]),
 array([[-73.98905588, -73.98905408],
        [-73.98905588, -73.98905408]]))

#### Step by step

In [110]:
BAND = dataset.GetRasterBand(1)
X_OFFSET = 150000
Y_OFFSET = 200000
X_SIZE = 1000
Y_SIZE = 50

#specify projection coordinate conversions
PROJ_FROM = pyproj.Proj(2263)
PROJ_TO = pyproj.Proj(proj = 'latlong')

In [111]:
#select sample array
# parameters: band, x offset, y offset, x size (array columns), y size (array rows)
lc_array = gdal_array.BandReadAsArray(BAND,
                                      X_OFFSET,
                                      Y_OFFSET,
                                      X_SIZE,
                                      Y_SIZE)
print(f'Land cover values in sample: {np.unique(lc_array)}')
print(f'Area of sample: {X_SIZE*Y_SIZE/4} sq ft')
print(f'Total pixels (observations) in sample: {X_SIZE*Y_SIZE}')

Land cover values in sample: [1 2 5 6 7]
Area of sample: 12500.0 sq ft
Total pixels (observations) in sample: 50000


In [112]:
# X_geo = GT(0) + X_pixel * GT(1) + Y_line * GT(2)
# Y_geo = GT(3) + X_pixel * GT(4) + Y_line * GT(5)
## gt(2),gt(4) = 0 so:
# x_geo = gt(0) + x_pixel*gt(1)
#y_geo = gt(3) + y_line*gt(5)

In [113]:
# gdal approach

#create meshgrid of equally spaced centroid points
cols, rows = np.meshgrid(np.arange(X_SIZE), np.arange(Y_SIZE))

#convert pixel to georef coordinates of pixel centroid, populate meshgrid
rc_en = lambda r,c : (geotransform[0] + (X_OFFSET + c + 0.5) * geotransform[1],
                      geotransform[3] + (Y_OFFSET + r + 0.5) * geotransform[5])

eastings, northings = np.vectorize(rc_en, otypes = [float, float])(rows, cols)

In [114]:
# transform LCC georef coordinates to lat long
transformer = pyproj.Transformer.from_proj(PROJ_FROM, PROJ_TO)
longs, lats = transformer.transform(eastings, northings)

In [115]:
# coordinates for pixel in lc_array[0][0]
lats[0][0],longs[0][0]

(40.64321750760775, -73.98905587738378)

Alternative option using `rasterio` instead of `gdal`:

In [2]:
# with rasterio.open('path/to/file.img') as r:
#     T0 = r.transform
#     p1 = pyproj.Proj(r.crs)
# T1 = T0 * Affine.translation(0.5,0.5)

In [51]:
# rc_en = lambda r,c : (c,r) * T1
# cols, rows = np.meshgrid(np.arange(X_SIZE), np.arange(Y_SIZE))

In [60]:
# # rasterio
# lats[0][0],longs[0][0]

## Converting coordinate and dimensions to pixel subset of img array

In [120]:
def get_coords_from_lat_long(lat, long, proj_from = pyproj.Proj(2263), proj_to = pyproj.Proj(proj = 'latlong')):
    
    transformer = pyproj.Transformer.from_proj(proj_to, proj_from)
    
    eastings, northings = transformer.transform(long, lat)
    
    x = int((eastings - geotransform[0])/geotransform[1]-0.5)
    y = int((northings - geotransform[3])/geotransform[5]-0.5)
    
    return x, y

def get_array_lat_long_dim(band, lat, long, length, height):
    
    x_offset, y_offset = get_coords_from_lat_long(lat, long)
    
    lc_array = gdal_array.BandReadAsArray(band, x_offset, y_offset, length*2, height*2)
    
    return lc_array

### Example

In [121]:
# get coordinates
lats, longs = get_lat_long_arrays(BAND,2,2,150000,100000)
print('Latitudes:',lats)
print('Longitudes:',longs)

Land cover values in sample: [4]
Area of sample: 1.0 sq ft
Total pixels (observations) in sample: 4
Latitudes: [[40.78045558 40.78045558]
 [40.78045421 40.78045421]]
Longitudes: [[-73.98903332 -73.98903152]
 [-73.98903332 -73.98903152]]


In [122]:
# return top left corner : (0,0) or offset
get_coords_from_lat_long(lats[0][0],longs[0][0])

(150000, 100000)

In [128]:
# get array of specified size with lat long in top left
# ex get array for 2ft x 3ft rectangle with (40.78045558,-73.98903332) in top left
print(get_array_lat_long_dim(band, lats[0][0],longs[0][0],2,3))
print('\n')
print(get_array_lat_long_dim(band, np.array(40.78045558), np.array(-73.98903332),2,3))

[[4 4 4 4]
 [4 4 4 4]
 [4 4 4 4]
 [4 4 4 4]
 [4 4 4 4]
 [4 4 4 4]]


[[4 4 4 4]
 [4 4 4 4]
 [4 4 4 4]
 [4 4 4 4]
 [4 4 4 4]
 [4 4 4 4]]
